In [17]:
import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import *
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
%matplotlib inline

In [3]:
batch_size = 4

train_datagen = ImageDataGenerator(
        rescale=1./255,    
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'data',  # this is the target directory
        target_size=(124, 124),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

Found 734 images belonging to 2 classes.


In [11]:
x, y = next(train_generator)
x.shape, y.shape

((4, 124, 124, 3), (4,))

In [13]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(124, 124, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/50
500/500 [==============================] - 88s 175ms/step - loss: 0.3886 - acc: 0.8135
Epoch 2/50
 41/500 [=>............................] - ETA: 1:34 - loss: 0.4223 - acc: 0.8415- ETA: 1:33 - loss: 0.471

In [37]:
x, y = next(train_generator)
y = y.reshape(len(y), 1)


y_pred = model.predict(x)
y_pred = (y_pred > 0.5) * 1

y == y_pred

((4, 124, 124, 3), (4, 1))

In [ ]:
img_path = random.choice(os.listdir('data/1'))
img = load_img("data/1/" + img_path) #this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)